In [9]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

In [2]:
#read file
path = 'C:/Users/waghpr02/Desktop/Projects/classification'
file = path+'/loan_pred_train.csv'
data = pd.read_csv(file)
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [3]:
data_oroginal = data.copy()
data.info()
data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
Loan_ID              614 non-null object
Gender               601 non-null object
Married              611 non-null object
Dependents           599 non-null object
Education            614 non-null object
Self_Employed        582 non-null object
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null float64
LoanAmount           592 non-null float64
Loan_Amount_Term     600 non-null float64
Credit_History       564 non-null float64
Property_Area        614 non-null object
Loan_Status          614 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 62.4+ KB


Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [4]:
#missing data imputations
mode_gender = data['Gender'].mode()
data['Gender'] = data['Gender'].fillna('Male')
data['Dependents'].mode()
data['Dependents'] = data['Dependents'].fillna(0)
data['Married'].mode()
data['Married'] = data['Married'].fillna('Yes')
data['Self_Employed'].mode()
data['Self_Employed'] = data['Self_Employed'].fillna('No')
data['LoanAmount'].mean()
data['LoanAmount'] = data['LoanAmount'].fillna(146)
data['Loan_Amount_Term'].median()
data['Loan_Amount_Term'] = data['Loan_Amount_Term'].fillna(360)
data['Credit_History'].mode()
data['Credit_History'] = data['Credit_History'].fillna(1)
data.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [10]:
#Balancing data using SMOTE
data['Loan_Status'].value_counts()
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
data_1 = data.copy()
data['Dependents'] = data['Dependents'].replace('3+',4)
lb = sklearn.preprocessing.LabelBinarizer()
le = sklearn.preprocessing.LabelEncoder()
data = data.drop('Loan_ID', axis = 1)
x = data.drop('Loan_Status', axis = 1)
y = data['Loan_Status']
y = lb.fit_transform(y)
x.columns
print(x['Gender'].value_counts())
print(x['Married'].value_counts())
print(x['Education'].value_counts())
print(x['Self_Employed'].value_counts())
print(x['Property_Area'].value_counts())

x['Gender'] = lb.fit_transform(x['Gender'])
x['Married'] = lb.fit_transform(x['Married'])
x['Education'] = lb.fit_transform(x['Education'])
x['Self_Employed'] = lb.fit_transform(x['Self_Employed'])
x['Property_Area'] = le.fit_transform(x['Property_Area'])

data_1['Loan_Status'].value_counts()

Male      502
Female    112
Name: Gender, dtype: int64
Yes    401
No     213
Name: Married, dtype: int64
Graduate        480
Not Graduate    134
Name: Education, dtype: int64
No     532
Yes     82
Name: Self_Employed, dtype: int64
Semiurban    233
Urban        202
Rural        179
Name: Property_Area, dtype: int64


Y    422
N    192
Name: Loan_Status, dtype: int64

In [12]:
sm = SMOTE(random_state = 22)
x_train,  x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 22)

x_train_res, y_train_res = sm.fit_sample(x_train, y_train.ravel())
print(sum(y_train_res==1))
print(sum(y_train_res==0))

338
338


In [13]:
#Logistic regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1, max_iter = 1000,random_state=22)
logreg = lr.fit(x_train_res, y_train_res)
y_pred = logreg.predict(x_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(x_test, y_test)))

from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

Accuracy of logistic regression classifier on test set: 0.75
[[21 18]
 [13 71]]
             precision    recall  f1-score   support

          0       0.62      0.54      0.58        39
          1       0.80      0.85      0.82        84

avg / total       0.74      0.75      0.74       123



In [14]:
#LogisticRegressionCV
from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(Cs=10, cv=3, random_state=22)
logreg_cv = clf.fit(x_train_res, y_train_res)
y_pred_CV = logreg_cv.predict(x_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg_cv.score(x_test, y_test)))

from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_CV)
print(confusion_matrix)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_CV))

Accuracy of logistic regression classifier on test set: 0.74
[[22 17]
 [15 69]]
             precision    recall  f1-score   support

          0       0.59      0.56      0.58        39
          1       0.80      0.82      0.81        84

avg / total       0.74      0.74      0.74       123



In [15]:
#GridsearchCV
from sklearn.grid_search import GridSearchCV
grid = {'C':np.logspace(-1,1,10),'penalty':['l1','l2']}

gscv = GridSearchCV(estimator = LogisticRegression(),param_grid = grid, cv = 10)
logreg_gscv = gscv.fit(x_train_res, y_train_res)
y_pred_gscv = logreg_gscv.predict(x_test)

from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_gscv)
print(confusion_matrix)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_gscv))

print("tuned hpyerparameters :(best parameters) ",gscv.best_params_)
print("Accuracy of GridsearchCV classifier on test set :",logreg_gscv.score(x_test, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


[[21 18]
 [ 9 75]]
             precision    recall  f1-score   support

          0       0.70      0.54      0.61        39
          1       0.81      0.89      0.85        84

avg / total       0.77      0.78      0.77       123

tuned hpyerparameters :(best parameters)  {'C': 0.16681005372000587, 'penalty': 'l1'}
Accuracy of GridsearchCV classifier on test set : 0.780487804878


In [16]:
#tree based model
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
clf_gini = DecisionTreeClassifier(criterion = 'gini',max_depth = 3, min_samples_leaf = 3)
decision_tree_clf_gini = clf_gini.fit(x_train_res, y_train_res)
y_pred_clf_gini = decision_tree_clf_gini.predict(x_test)

from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_clf_gini)
print(confusion_matrix)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_clf_gini))

print("Accuracy of Decision Tree(GINI) classifier on test set :",decision_tree_clf_gini.score(x_test, y_test))

# In[6]
# Tree Based Model = Entropy
from sklearn.tree import DecisionTreeClassifier
clf_entropy = DecisionTreeClassifier(criterion = 'entropy',max_depth = 3, min_samples_leaf = 3)
decision_tree_clf_entropy = clf_entropy.fit(x_train_res, y_train_res)
y_pred_clf_entropy = decision_tree_clf_entropy.predict(x_test)

from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_clf_entropy)
print(confusion_matrix)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_clf_entropy))

print("Accuracy of Decision Tree(entropy) classifier on test set :",decision_tree_clf_entropy.score(x_test, y_test))

[[18 21]
 [ 0 84]]
             precision    recall  f1-score   support

          0       1.00      0.46      0.63        39
          1       0.80      1.00      0.89        84

avg / total       0.86      0.83      0.81       123

Accuracy of Decision Tree(GINI) classifier on test set : 0.829268292683
[[18 21]
 [ 0 84]]
             precision    recall  f1-score   support

          0       1.00      0.46      0.63        39
          1       0.80      1.00      0.89        84

avg / total       0.86      0.83      0.81       123

Accuracy of Decision Tree(entropy) classifier on test set : 0.829268292683


In [17]:
#RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf_model =  rf.fit(x_train_res, y_train_res)
y_pred_rf = rf_model.predict(x_test)

from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_rf)
print(confusion_matrix)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_rf))

print("Accuracy of RandomForestClassifier classifier on test set :",rf_model.score(x_test, y_test))

[[23 16]
 [13 71]]
             precision    recall  f1-score   support

          0       0.64      0.59      0.61        39
          1       0.82      0.85      0.83        84

avg / total       0.76      0.76      0.76       123

Accuracy of RandomForestClassifier classifier on test set : 0.764227642276
